# Initializing project

Install and load Python dependencies

In [3]:
from tensorflow import keras
from keras.layers import *
from keras.models import *

import os
import math
import numpy as np
import cv2
import tensorflow as tf

from tqdm import tqdm
from google.colab import files
from google.colab import drive
from IPython.display import clear_output
from IPython.display import display

Download and unzip the DIV2K [dataset](https://data.vision.ee.ethz.ch/cvl/DIV2K/)

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip

In [ ]:
!unzip "DIV2K_train_HR.zip"
clear_output()

OR get dataset from drive

In [ ]:
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/School/2022-2023/Winter/ELECENG 4TN4/sr_cnn/DIV2K_train_HR.zip"
clear_output()
drive.flush_and_unmount()

# Define the model


In [ ]:
def get_model(scale):
  kernel_size = 3
  conv_args = {
    "activation": "relu",
    "padding": "same",
  }

  input = Input(shape=(None, None, 1))

  input = input / 255

  x = Conv2D(32, kernel_size, **conv_args)(input)
  x = Conv2D(64, kernel_size, **conv_args)(x)
  x = Conv2D(128, kernel_size, **conv_args)(x)
  x = UpSampling2D(scale)(x)
  x = Conv2D(64, kernel_size, **conv_args)(x)
  x = Conv2D(32, kernel_size, **conv_args)(x)
  x = Conv2D(1, kernel_size, activation=None, padding='same')(x)
  x = Activation('tanh')(x)
  x = x * 127.5 + 127.5

  model = Model([input], x)

  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
  model.compile(loss='mse', optimizer=optimizer)
  model.summary()
  
  return model

# Processing Data

In [2]:
class Encoder:
  def __init__(self, image):
    self.image = image
    self.ycbcr = self.rgb2ycbcr(cv2.split(image))

  def encode(self):
    '''
    Downsamples image in YCbCr colorspace
    Returns: Downsampled Y, Cb, Cr channels as tuple
    '''
    B, G, R = cv2.split(self.image)

    ycbcr = self.rgb2ycbcr((R, G, B))

    # Downsample Y by a factor of 2
    Y = self.downsample(ycbcr[0], 2)

    # Downsample Cb and Cr by a factor of 4
    Cb = self.downsample(ycbcr[1], 4)
    Cr = self.downsample(ycbcr[2], 4)

    self.ycbcr = (Y, Cb, Cr)
    
  def rgb2ycbcr(self, rgb):
    '''
    Convert RGB colorspace to YCbCr colorspace
    Parameters: R, G, B channels
    Returns: Y, Cb, Cr channels as tuple
    '''
    R, G, B = rgb
    Y = 0.257*R + 0.504*G + 0.098*B + 16
    Cb = -0.148*R - 0.291*G + 0.439*B + 128
    Cr = 0.439*R - 0.368*G - 0.071*B + 128

    Y = np.clip(Y, 0, 255).astype(np.uint8)
    Cb = np.clip(Cb, 0, 255).astype(np.uint8)
    Cr = np.clip(Cr, 0, 255).astype(np.uint8)

    return (Y, Cb, Cr)

  def downsample(self, channel, factor):
    return channel[::factor, ::factor]
    
  def get_ycbcr(self):
    return self.ycbcr

In [ ]:
def get_data():
  x_Y = []
  x_Cb = []
  x_Cr = []
  y_Y = []
  y_Cb = []
  y_Cr = []

  imgs = os.listdir("./DIV2K_train_HR")

  for img_path in tqdm(imgs):
    img = cv2.imread(f"./DIV2K_train_HR/{img_path}", 1)
    B, G, R = cv2.split(img)

    encoder = Encoder(img)
    ycbcr = encoder.rgb2ycbcr((R, G, B))
    y_channel, Cb_channel, Cr_channel = ycbcr

    Y_out = cv2.resize(y_channel, (256, 256), interpolation=cv2.INTER_AREA)
    Cb_out = cv2.resize(Cb_channel, (256, 256), interpolation=cv2.INTER_AREA)
    Cr_out = cv2.resize(Cr_channel, (256, 256), interpolation=cv2.INTER_AREA)

    Y_in = encoder.downsample(Y_out, 2)
    Cb_in = encoder.downsample(Cb_out, 4)
    Cr_in = encoder.downsample(Cr_out, 4)

    x_Y.append(Y_in)
    x_Cb.append(Cb_in)
    x_Cr.append(Cr_in)
    y_Y.append(Y_out)
    y_Cb.append(Cb_out)
    y_Cr.append(Cr_out)

  x_Y = np.array(x_Y)
  x_Cb = np.array(x_Cb)
  x_Cr = np.array(x_Cr)
  y_Y = np.array(y_Y)
  y_Cb = np.array(y_Cb)
  y_Cr = np.array(y_Cr)

  return x_Y, x_Cb, x_Cr, y_Y, y_Cb, y_Cr

In [ ]:
Y_model = get_model(2)
Cb_model = get_model(4)
Cr_model = get_model(4)
x_Y, x_Cb, x_Cr, y_Y, y_Cb, y_Cr = get_data()

# import matplotlib.pyplot as plt
# plt.subplot(221); plt.imshow(x[0], cmap="gray")
# plt.subplot(222); plt.imshow(y[0], cmap="gray")
# plt.show()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    320       
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    18496     
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 128)   73856     
                                                                 
 up_sampling2d (UpSampling2D  (None, None, None, 128)  0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 64)    73792     
                                                             

100%|██████████| 800/800 [03:55<00:00,  3.40it/s]


# Training model

In [ ]:
from sklearn.model_selection import train_test_split

batch_size = 4
epochs = 100

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
def tbCallback(model):
  return tf.keras.callbacks.TensorBoard(log_dir=f'./Graph/{model}/', histogram_freq=0, write_graph=True, write_images=True)
def save_model_callback(model):
  return tf.keras.callbacks.ModelCheckpoint(
      f'model/{model}.h5',
      monitor='val_loss',
      verbose=1,
      save_best_only=True,
      mode='min',
      save_freq='epoch'
  )

x_train, x_val, y_train, y_val = train_test_split(x_Y, y_Y, test_size=0.2, random_state=42)
print("\n\nTraining Y channel model... \n")
Y_model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[save_model_callback('Y_model'), tbCallback('Y_model'), stop_early])

x_train, x_val, y_train, y_val = train_test_split(x_Cb, y_Cb, test_size=0.2, random_state=42)
print("\n\nTraining Cb channel model... \n")
Cb_model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[save_model_callback('Cb_model'), tbCallback('Cb_model'), stop_early])

x_train, x_val, y_train, y_val = train_test_split(x_Cr, y_Cr, test_size=0.2, random_state=42)
print("\n\nTraining Cr channel model... \n")
Cr_model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[save_model_callback('Cr_model'), tbCallback('Cr_model'), stop_early])





Training Y channel model... 

Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 3594.9961
Epoch 1: val_loss improved from inf to 2348.71021, saving model to model/Y_model.h5
160/160 [==============================] - 22s 52ms/step - loss: 3594.9961 - val_loss: 2348.7102
Epoch 2/100
159/160 [============================>.] - ETA: 0s - loss: 2329.2424
Epoch 2: val_loss improved from 2348.71021 to 2059.24023, saving model to model/Y_model.h5
160/160 [==============================] - 8s 50ms/step - loss: 2324.9373 - val_loss: 2059.2402
Epoch 3/100
159/160 [============================>.] - ETA: 0s - loss: 1942.5466
Epoch 3: val_loss improved from 2059.24023 to 1842.62329, saving model to model/Y_model.h5
160/160 [==============================] - 8s 51ms/step - loss: 1937.1549 - val_loss: 1842.6233
Epoch 4/100
159/160 [============================>.] - ETA: 0s - loss: 1737.8402
Epoch 4: val_loss improved from 1842.62329 to 1652.60840, saving model to model/Y_model.h

# Save model and predict




In [ ]:
!zip -r ./Graph.zip ./Graph

In [ ]:
files.download('./model/Y_model.h5')
files.download('./model/Cb_model.h5')
files.download('./model/Cr_model.h5')
files.download('./Graph.zip')

In [36]:
class Decoder:
    def __init__(self, ycbcr):
        self.ycbcr = ycbcr

    def decode(self, size):
        """
        Upsamples image in YCbCr colorspace
        Parameters: Size of image to upsample to
        Returns: Reconstructed image
        """
        Y, Cb, Cr = self.infer()

        R, G, B = self.ycbcr2rgb((Y, Cb, Cr))

        self.image = cv2.merge((B, G, R))
        return cv2.merge((B, G, R))

    def ycbcr2rgb(self, ycbcr):
        """
        Convert YCbCr colorspace to RGB colorspace
        Parameters: Y, Cb, Cr channels
        Returns: R, G, B channels as tuple
        """
        Y, Cb, Cr = ycbcr
        R = 1.164 * (Y - 16) + 1.596 * (Cr - 128)
        G = 1.164 * (Y - 16) - 0.392 * (Cb - 128) - 0.813 * (Cr - 128)
        B = 1.164 * (Y - 16) + 2.017 * (Cb - 128)

        R = np.clip(R, 0, 255).astype(np.uint8)
        G = np.clip(G, 0, 255).astype(np.uint8)
        B = np.clip(B, 0, 255).astype(np.uint8)

        return (R, G, B)

    def infer(self):
        Y_model = load_model("model/Y_model.h5")
        Cb_model = load_model("model/Cb_model.h5")
        Cr_model = load_model("model/Cr_model.h5")

        Y, Cb, Cr = self.ycbcr
        y_Y = np.expand_dims(Y, axis=0)
        y_Cb = np.expand_dims(Cb, axis=0)
        y_Cr = np.expand_dims(Cr, axis=0)

        Y = Y_model.predict(y_Y)
        Cb = Cb_model.predict(y_Cb)
        Cr = Cr_model.predict(y_Cr)

        Y = Y[0,:,:,0]
        Cb = Cb[0,:,:,0]
        Cr = Cr[0,:,:,0]

        return (Y, Cb, Cr)

    def get_psnr(self, original_image):
        mse = np.mean((original_image - self.image) ** 2)
        return 10 * math.log10(255.0**2 / mse)

    def get_image(self):
        return self.image

In [ ]:
def main():
    PATH = "birds.jpg"
    FILENAME = PATH.split("/")[-1].split(".")[0]
    EXTENSION = PATH.split(".")[-1]
    original_image = cv2.imread(PATH, 1)

    encoder = Encoder(original_image)
    encoder.encode()
    downsampled_image = encoder.get_ycbcr()

    decoder = Decoder(downsampled_image)
    decoder.decode(original_image.shape)
    final_image = decoder.get_image()

    psnr = decoder.get_psnr(original_image)
    print("PSNR:", psnr)

    cv2.imwrite(f"{FILENAME}_upsampled.{EXTENSION}", final_image)

if __name__ == "__main__":
    main()
